# Análisis de preferencias musicales por grupo demográfico

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Observaciones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypothesis)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>

El objetivo de este proyecto es analizar un conjunto de datos para extraer información relevante que permita tomar decisiones informadas. Para ello, se abordarán varias etapas, como la revisión inicial de los datos, el preprocesamiento y la prueba de hipótesis.

En este caso, se compararán las preferencias musicales de los usuarios de las ciudades de **Springfield** y **Shelbyville**. Utilizando datos reales de reproducción de música en línea, se evaluarán hipótesis relacionadas con el comportamiento musical por día de la semana y por ciudad.

### Objetivo

Probar la siguiente hipótesis:

1. La actividad de los usuarios varía según el día de la semana y depende de la ciudad de residencia.

### Etapas del proyecto

Los datos de comportamiento del usuario se encuentran en el archivo `/datasets/music_project_en.csv`.

El proyecto se divide en las siguientes etapas:

1. Descripción de los datos  
2. Preprocesamiento  
3. Prueba de hipótesis

## Etapa 1. Descripción de los datos <a id='data_review'></a>

In [2]:
# Importar pandas
import pandas as pd

In [3]:
# Leer el archivo y almacenarlo en df
df = pd.read_csv("/datasets/music_project_en.csv")

Se muestran las 10 primeras filas de la tabla, para observar la estructura de los datos y las columnas que contiene.

In [4]:
# Obtener las 10 primeras filas de la tabla df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [5]:
# Obtener la información general sobre nuestros datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Estas son nuestras observaciones sobre la tabla. Contiene siete columnas. Almacenan los mismos tipos de datos: `object`.

Según la documentación del conjunto de datos, las columnas representan:

- `' userID'`: identificador del usuario  
- `'Track'`: título de la canción  
- `'artist'`: nombre del artista  
- `'genre'`: género musical  
- `'City'`: ciudad del usuario  
- `'time'`: hora exacta de reproducción  
- `'Day'`: día de la semana  

Se identifican algunos problemas en los encabezados:

1. Uso inconsistente de mayúsculas y minúsculas. 
2. Presencia de espacios innecesarios.
3. Encabezados poco descriptivos.

Aunque todos los datos son categóricos, es importante analizar los valores únicos en cada categoría.

Con aproximadamente 65,000 registros, el volumen de datos es suficiente para realizar un análisis fiable. Las columnas clave para el estudio de la hipótesis (`'City'`, `'time'` y `'Day'`) no presentan valores ausentes, lo cual es ideal. Si bien existen algunos valores faltantes en otras columnas, no representan un porcentaje significativo.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

El objetivo aquí es preparar los datos para que sean analizados.
El primer paso es resolver cualquier problema con los encabezados. Luego podemos avanzar a los valores ausentes y duplicados.

### Estilo del encabezado <a id='header_style'></a>

In [6]:
# Muestra los nombres de las columnas
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Se adaptarán los encabezados a estas reglas de estilo:
* Todos los caracteres deben ser minúsculas.
* Se elimina los espacios.
* Si el nombre tiene varias palabras, se utiliza snake_case.

Se utilizará el bucle for para iterar sobre los nombres de las columnas y poner todos los caracteres en minúsculas.

In [7]:
# Bucle en los encabezados poniendo todo en minúsculas
new_col = []

for name in df.columns:
    name = name.lower()    
    new_col.append(name)

df.columns = new_col
print(df.columns)

Index(['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day'], dtype='object')


Ahora, utilizando el mismo método, se eliminan los espacios al principio y al final de los nombres de las columnas:

In [8]:
# Bucle en los encabezados eliminando los espacios
new_col = []

for name in df.columns:    
    name = name.strip()
    new_col.append(name)

df.columns = new_col
print(df.columns)

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


Necesitamos aplicar la regla de snake_case a la columna `userid`. Debe ser `user_id`.

In [9]:
# Cambiar el nombre de la columna "userid"
df.rename(columns={'userid': 'user_id'}, inplace=True)

print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Valores ausentes <a id='missing_values'></a>
 Primero, se encuentra el número de valores ausentes en la tabla.

In [11]:
# Calcular el número de valores ausentes
print(df.isna().sum())

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en `track` y `artist` no son cruciales. Simplemente se pueden reemplazar con valores predeterminados como el string `'unknown'` (desconocido).

Pero los valores ausentes en `'genre'` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. Ya que no conocemos las razones por las cuales hay datos ausentes en este proyecto con fines prácticos rellenaremos estos valores ausentes con un valor predeterminado.

Reemplazaremos los valores ausentes en las columnas `'track'`, `'artist'` y `'genre'` con el string `'unknown'`. La mejor forma de hacerlo es crear una lista que almacene los nombres de las columnas donde se necesita el reemplazo. Luego, utilizar esta lista e iterar sobre las columnas donde se necesita el reemplazo haciendo el propio reemplazo.

In [12]:
# Bucle en los encabezados reemplazando los valores ausentes con 'unknown'
col_names = ['track', 'artist', 'genre']

for col in col_names:
    df[col].fillna('unknown', inplace=True)

 Ahora se comprueba el resultado para asegurarse de que después del reemplazo no haya valores ausentes en el conjunto de datos.

In [13]:
# Contar valores ausentes
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicados <a id='duplicates'></a>

In [14]:
# Contar duplicados explícitos
print(df.duplicated().sum())

3826


Ahora, se eliminan todos los duplicados.

In [15]:
# Eliminar duplicados explícitos
df = df.drop_duplicates().reset_index(drop=True)

In [16]:
# Comprobar de nuevo si hay duplicados
print(df.duplicated().sum())


0


Ahora queremos deshacernos de los duplicados implícitos en la columna `genre`. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar al resultado.

Para hacerlo, primero mostraremos una lista de nombres de género únicos, ordenados en orden alfabético. Para ello:
* Se extrae la columna `genre` del DataFrame.
* Se llama al método que devolverá todos los valores únicos en la columna extraída.


In [ ]:
# Inspeccionar los nombres de géneros únicos
print(sorted(df['genre'].unique()))
print('Total :', df['genre'].nunique())

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'film', 'fitness', 'flamenco', 'folk', 'folklor

Se crea una función llamada `replace_wrong_genres()` con dos parámetros:
* `wrong_genres=`: esta es una lista que contiene todos los valores que se reemplazan.
* `correct_genre=`: este es un string que se utiliza como reemplazo.

Como resultado, la función debería corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, remplazar cada valor de la lista `wrong_genres` por el valor en `correct_genre`.

In [19]:
# Función para reemplazar duplicados implícitos
def replace_wrong_genres(wrong_genres, correct_genre):
    for name in wrong_genres:
        df[df['genre'] == name] = df[df['genre'] == name].replace(name, correct_genre)

Ahora, se llama a `replace_wrong_genres()` y pasandole los argumentos para que retire los duplicados implícitos (`hip`, `hop` y `hip-hop`) y los reemplace por `hiphop`:

In [20]:
# Eliminar duplicados implícitos
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')

In [ ]:
# Comprobación de duplicados implícitos
print(sorted(df['genre'].unique()))
print('Total :', df['genre'].nunique())

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'film', 'fitness', 'flamenco', 'folk', 'folklor

### Observaciones <a id='data_preprocessing_conclusions'></a>
- Los encabezados de la tabla se han adaptado a las reglas de estilo. Se eliminaron los espacios y se utilizaron letras minúsculas. La columna `userid` se ha renombrado a `user_id`.
- Los valores ausentes se han reemplazado por el string `'unknown'` en las columnas `track`, `artist` y `genre`. Después de este paso, no hay valores ausentes en la tabla.
- Habia una gran cantidad de duplicados, para abordar su eliminacion se recurrío al metodo drop_duplicates() junto con reset_index() para poder tener la tabla sin duplicados con una indexación ordenada nuevamente.

## Etapa 3. Prueba de hipótesis <a id='hypothesis'></a>

La hipótesis afirma que existen diferencias en la forma en que los usuarios y las usuarias de Springfield y Shelbyville consumen música. Para comprobar esto, se usará los datos de tres días de la semana: lunes, miércoles y viernes. El proceso se describe acontinuación:

* Se agrupa a los usuarios y las usuarias por ciudad.
* Se compara el número de canciones que cada grupo reprodujo el lunes, el miércoles y el viernes.


El objetivo ahora es agrupar los datos por ciudad, aplicar el método apropiado para contar durante la etapa de aplicación y luego encontrar la cantidad de canciones reproducidas en cada grupo especificando la columna para obtener el recuento.

In [22]:
# Contar las canciones reproducidas en cada ciudad
print(df.groupby('city')['genre'].count())

city
Shelbyville    18512
Springfield    42741
Name: genre, dtype: int64


Springfield tiene mas de 2 veces la cantidad de reproducciones que Shelbyville, eso significa que los resultados de las pruebas tendran más peso para la poblacion de Springfield.

Ahora agrupemos los datos por día de la semana y encontremos el número de canciones reproducidas el lunes, miércoles y viernes.

In [23]:
# Calcular las canciones reproducidas en cada uno de los tres días
print(df.groupby('day')['genre'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64


A medida que la semana avanza las personas suelen reproducir mas música.

Crearemos la función `number_tracks()` para calcular el número de canciones reproducidas en un determinado día **y** ciudad. La función debe aceptar dos parámetros:

- `day`: un día de la semana para filtrar. Por ejemplo, `'Monday'` (lunes).
- `city`: una ciudad para filtrar. Por ejemplo, `'Springfield'`.

Después de filtrar los datos por dos criterios, cuenta el número de valores de la columna 'user_id' en la tabla resultante.

In [ ]:
# Función number_tracks() con dos parámetros: day= y city=.
def number_tracks(day, city):
    # Almacena las filas del DataFrame donde el valor en la columna 'day' es igual al parámetro day= y en la columna 'city' es igual al parámetro city=
    df_day_city = df[(df['day'] == day) & (df['city'] == city)]
    # Extrae la columna 'user_id' de la tabla filtrada y aplica el método count()
    users_amount = df_day_city['user_id'].count()
    # Devolve el número de valores de la columna 'user_id'
    return users_amount

Llamaremos a `number_tracks()` seis veces, cambiando los valores de los parámetros para que recuperes los datos de ambas ciudades para cada uno de los tres días.

In [25]:
# El número de canciones reproducidas en Springfield el lunes
print(number_tracks('Monday', 'Springfield'))

15740


In [26]:
# El número de canciones reproducidas en Shelbyville el lunes
print(number_tracks('Monday', 'Shelbyville'))

5614


In [27]:
# El número de canciones reproducidas en Springfield el miércoles
print(number_tracks('Wednesday', 'Springfield'))

11056


In [28]:
# El número de canciones reproducidas en Shelbyville el miércoles
print(number_tracks('Wednesday', 'Shelbyville'))

7003


In [29]:
# El número de canciones reproducidas en Springfield el viernes
print(number_tracks('Friday', 'Springfield'))

15945


In [30]:
# El número de canciones reproducidas en Shelbyville el viernes
print(number_tracks('Friday', 'Shelbyville'))

5895


# Conclusiones <a id='end'></a>

A pesar de los resultados obtenidos, recomendamos más estudios para poder argumentar que la hipótesis es fuertemente sostenida por los datos, aunque podemos decir que la hipótesis es parcialmente cierta con estas observaciones.

Los resultados de las pruebas muestran que el número de reproducciones de canciones en Springfield es mayor que en Shelbyville. En Springfield, el lunes se reproducen 2.8 veces más canciones que en Shelbyville, el miércoles 1,5 veces más y el viernes 2,7 veces más. Concluimos también que las ciudades muestran diferentes patrones a lo largo de la semana. Mientras Springfield tiene más reproducciones a inicio y final de semana (lunes y viernes), Shelbyville encuentra el mayor número de reproducciones a mitad de semana (miércoles).